# Harmony Feature Examples

This notebook provides condensed examples of using Harmony to perform specific tasks.  For more a general introduction and tutorial, see [Harmony API Introduction](./Harmony%20Api%20Introduction.ipynb).  Useful helpers for making the calls found in this note book can be found under the [docs/notebook-helpers](./notebook-helpers) folder

## Prerequisites

1. Install Python 3.  This notebook is tested to work in 3.8 but should work in most recent 3.x versions.
2. Install Jupyter: `pip install jupyterlab`
3. Setup your `~/.netrc` for Earthdata Login as described in [Harmony API Introduction](./Harmony%20Api%20Introduction.ipynb)
4. Run the following cell to install Python dependencies, import necessary modules, and set notebook defaults

In [ ]:
%load_ext autoreload
%autoreload

import sys
# Install dependencies into the Jupyter Kernel
!{sys.executable} -m pip install -q -r notebook_helpers/requirements.txt
!{sys.executable} -m pip install s3fs zarr

%matplotlib inline
# Import libraries used throughout the notebook
from notebook_helpers import get, post, show, get_data_urls, show_async, show_async_condensed, show_shape, print_async_status

## Example Data

Harmony has produced example collections with artificial data but realistic structure to allow testing our services.  We have L3 and L2 NetCDF4 collections, and a shapefile collection.

In [ ]:
l3_collection = 'C1234088182-EEDTEST'
l2_collection = 'C1233860183-EEDTEST'

In [ ]:
response = get(f'https://harmony.uat.earthdata.nasa.gov/service-results/harmony-uat-eedtest-data/{l3_collection}/nc/2020_01_16_ff0000_global.nc')
show(response)

## OGC API - Coverages Requests

In [ ]:
coverages_root = 'https://harmony.uat.earthdata.nasa.gov/{collection}/ogc-api-coverages/1.0.0/collections/{variable}/coverage/rangeset'

### Bounding Box and Temporal Subsetting with Reformatting

In [ ]:
params = {
    'subset': [
        'lon(-20:90)', 
        'lat(0:60)', 
        'time("2020-01-15T00:00:00Z":"2020-01-15T01:00:00Z")']
}
response = get(
    coverages_root.format(
        collection=l3_collection, 
        variable='all'), 
    params=params)

show(response)

### Variable Subsetting

In [ ]:
response = get(
    coverages_root.format(
        collection=l3_collection, 
        variable='green_var'), 
    params=params)

show(response, color_index=1)

### Shapefile Subsetting

In [ ]:
show_shape('./notebook_helpers/namibia.geo.json')

#### CMR Search

In [ ]:
cmr_params = { 
    'collection_concept_id': l3_collection,
    'temporal': '2020-02-16T00:00:00Z,',
    'page_size': 150}
response = post('https://cmr.uat.earthdata.nasa.gov/search/granules.json', params=cmr_params)
[e['title'] for e in response.json()['feed']['entry']]

In [ ]:
response = post(
    'https://cmr.uat.earthdata.nasa.gov/search/granules.json', 
    files={ 'shapefile': ('namibia.geo.json', open('./notebook_helpers/namibia.geo.json', 'r'), 'application/geo+json') },
    data=cmr_params)
[e['title'] for e in response.json()['feed']['entry']]

In [ ]:
response = post(
    coverages_root.format(
        collection=l3_collection, 
        variable='all'), 
    data={ 'subset': 'time("2020-02-16T00:00:00Z":*)' },
    files={ 'shapefile': ('namibia.geo.json', open('./notebook_helpers/namibia.geo.json', 'r'), 'application/geo+json') })

show(response)

### Regridding 

#### L2 Test Data

In [ ]:
response = get(f'https://harmony.uat.earthdata.nasa.gov/service-results/harmony-uat-eedtest-data/{l2_collection}/nc/015_02_210_europe.nc')
show(response)

#### Basic Regridding

In [ ]:
response = get(
    coverages_root.format(
        collection=l2_collection, 
        variable='all'), 
    params={
        'outputCrs': 'EPSG:4326',
        'subset': 'time("2020-01-15T16:00:00Z":"2020-01-15T17:00:00Z")'})
show(response)

#### Advanced Regridding

In [ ]:
response = get(
    coverages_root.format(
        collection=l2_collection, 
        variable='all'), 
    params={
        'outputCrs': '+proj=lcc +lat_1=43 +lat_2=62 +lat_0=30 +lon_0=10 +x_0=0 +y_0=0 +ellps=intl +units=m +no_defs',
        'interpolation': 'near',
        'scaleExtent': '-4000000,-1000000,5000000,7000000',
        'subset': 'time("2020-01-15T16:00:00Z":"2020-01-15T17:00:00Z")'})

show(response)

## Asynchronous Requests

### Multiple Output Files

In [ ]:
response = get(
    coverages_root.format(
        collection=l3_collection, 
        variable='all'), 
    params={
        'subset': [
            'lon(-20:90)', 
            'lat(0:60)', 
            'time("2020-01-01T00:00:00Z":"2020-01-05T01:00:00Z")']})
show_async(response)

### Zarr Reformatting

In [ ]:
response = get(
    coverages_root.format(
        collection=l3_collection, 
        variable='all'), 
    params={'subset': 'time("2020-01-15T00:00:00Z":"2020-01-16T01:00:00Z")'},
    headers = {'accept': 'application/x-zarr'})

zarr_response = show_async(response)

In [ ]:
# Note: This will only work if you are running this notebook in the same AWS region as Harmony with valid credentials
!{sys.executable} -m pip install -q 's3fs>=0.4.2' 'zarr>=2.4.0'

import s3fs
import zarr

urls = get_data_urls(zarr_response)
fs = s3fs.S3FileSystem(profile='uat')
store = fs.get_mapper(root=urls[0], check=False)
zarr_file = zarr.open(store)

print(zarr_file.tree())

In [ ]:
# Note: This will only work if you are running this notebook in the same AWS region as Harmony with valid credentials
import numpy as np
from matplotlib import pyplot as plt

image = np.dstack([np.flip(zarr_file[v][0,900:1500,1600:2700], 0) for v in ['red_var', 'green_var', 'blue_var', 'alpha_var']])
plt.imshow(image);

### STAC

In [ ]:
from satstac import Catalog

stac_urls = [link['href'] for link in zarr_response.json()['links'] if link.get('rel', 'data') == 'stac-catalog-json']
cat = Catalog.open(stac_urls[0])

for i in cat.items():
    print('STAC Item')
    print('\t', 'ID:', i.id)
    print('\t', 'Date:', i.datetime)
    print('\t', 'Bounding Box:', i.bbox)
    print('\t', 'File:', list(i.assets.keys()))


### Cancel Job

In [ ]:
#Add 3 requests
response1 = get(coverages_root.format(collection=l3_collection, variable='all'), params={'format': 'image/tiff'})
response2 = get(coverages_root.format(collection=l3_collection, variable='all'), params={'format': 'image/tiff'})
response3 = get(coverages_root.format(collection=l3_collection, variable='all'), params={'format': 'image/tiff'})

#List the requests
my_jobs = 'https://harmony.uat.earthdata.nasa.gov/jobs'
response = get(my_jobs,params={'page': '1','limit': '10'})
body = response.json()

for job in body['jobs']:
    print_async_status(job)

#Cancel one
my_jobs_cancel_root=my_jobs+'/{job_id}/cancel'
response = post(my_jobs_cancel_root.format(job_id=response3.json()['jobID']))

print_async_status(response.json())

assert response.json()['status'] == 'canceled'


In [ ]:
# Cancel out the two paused requests
response = post(my_jobs_cancel_root.format(job_id=response1.json()['jobID']))
print_async_status(response.json())
assert response.json()['status'] == 'canceled'

response = post(my_jobs_cancel_root.format(job_id=response2.json()['jobID']))
print_async_status(response.json())
assert response.json()['status'] == 'canceled'


_Minimizing time to science_